In [1]:
import pandas as pd
from sqlalchemy import create_engine
import getpass

In [2]:
p = getpass.getpass(prompt="Password: ")
rds_connection_string = f"postgres:{p}@localhost:5432/Hockey_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

Password: ········


## Game Info

In [3]:
filepath1 = "Resources/game.csv"
game_df = pd.read_csv(filepath1)

In [4]:
game_df = game_df[["game_id", "season", "type", "away_team_id", "home_team_id", "outcome"]]

In [5]:
game_df = game_df.loc[game_df["type"] == "R"]

In [6]:
game_df = game_df.loc[game_df["season"] == 20172018]

In [8]:
game_df.to_csv("Output/game_info_df.csv", index=False, header=True)

In [9]:
game_df.to_sql('games', con=engine, if_exists="append", index=False)

## Player Info

In [10]:
filepath2 = "Resources/player_info.csv"
player_info_df = pd.read_csv(filepath2, sep="|")

In [11]:
player_info_df = player_info_df.assign(name = player_info_df.firstName.astype(str) + " " +
                                                       player_info_df.lastName.astype(str))

In [12]:
player_info_df = player_info_df[["player_id", "name", "primaryPosition"]]

In [13]:
player_info_df = player_info_df.rename(columns={"primaryPosition":"primary_position"})

In [15]:
player_info_df.to_csv("Output/player_info_df.csv", index=False, header=True, sep=",")

In [16]:
player_info_df.to_sql('players', con=engine, if_exists="append", index=False)

## Team Info

In [17]:
filepath3 = "Resources/team_info.csv"
team_info_df = pd.read_csv(filepath3)

In [18]:
team_info_df = team_info_df.assign(team = team_info_df.shortName.astype(str) + " " +
                                                       team_info_df.teamName.astype(str))

In [19]:
team_info_df = team_info_df[["team_id", "team"]]

In [20]:
team_info_df = team_info_df.drop([18, 32])

In [21]:
team_info_df["team"] = team_info_df["team"].replace({"NY Rangers Rangers": "New York Rangers"
                                                   , "NY Islanders Islanders": "New York Islanders"})

In [23]:
team_info_df.to_csv("Output/team_info_df.csv", index=False, header=True)

In [24]:
team_info_df.to_sql('teams', con=engine, if_exists="append", index=False)

## Skater Stats

In [25]:
filepath4 = "Resources/game_skater_stats.csv"
skater_info_df = pd.read_csv(filepath4, sep="|")

In [26]:
season_skater_df = skater_info_df.loc[skater_info_df.game_id.isin(game_df.game_id)]

In [27]:
season_skater_df = season_skater_df.reset_index()[['game_id','player_id','team_id','penaltyMinutes']]

In [28]:
season_skater_df = season_skater_df.rename(columns={"penaltyMinutes":"penalty_minutes"})

In [29]:
season_skater_df.to_csv("Output/skater_info_df.csv", index=False, header=True, sep=",")

In [30]:
season_skater_df.to_sql("skater_stats", con=engine, if_exists="append", index=False)

## Merge game_info with team_info

In [31]:
new_game_df = pd.merge(game_df, team_info_df, left_on="away_team_id", right_on="team_id", how="left")

In [32]:
new_game_df = pd.merge(new_game_df, team_info_df, left_on="home_team_id", right_on="team_id", how="left")

In [33]:
new_game_df = new_game_df.rename(columns={"team_x": "away_team", "team_y": "home_team"})

In [34]:
new_game_df = new_game_df[["game_id", "season", "home_team_id", "home_team", "away_team_id" , "away_team", "outcome"]]

## Analysis - PIM (Penalty in Minutes) by Position per Game

In [35]:
# Combine Skater states with player info
player_stats_df = season_skater_df.merge(player_info_df,on="player_id",how="inner")

# Combine LW (left Wing) and RW (Right Wing) into W (Wing)
player_stats_df['primary_position'] = player_stats_df['primary_position'].replace({"LW":"W","RW":"W"})

# Aggregate total PIM by position for each game
game_position_pim = player_stats_df[['game_id','primary_position','penalty_minutes']].groupby(
    ['game_id','primary_position',]).sum()

# Reset Index
game_position_pim.reset_index(inplace=True)

# Average PIM by Position per game
avg_position_pim = game_position_pim[['primary_position','penalty_minutes']].groupby(
    ['primary_position']).mean().rename(columns={'penalty_minutes':'avg_pim_by_position/game'})

# Aggregate total PIM by position for each team and game
team_game_position_pim = player_stats_df[['game_id','team_id','primary_position','penalty_minutes']].groupby(
    ['game_id','team_id','primary_position',]).sum()

# Reset Index
team_game_position_pim.reset_index(inplace=True)

# Calculate average PIM by Position
game_position_pim = team_game_position_pim.merge(avg_position_pim, on="primary_position")

In [38]:
# Combine PIM by Position per game with Game data by Home Team
game_results = new_game_df[['game_id','home_team_id','home_team','away_team_id','away_team','outcome']].merge(
    game_position_pim.rename(columns={'primary_position':'home_team_position','penalty_minutes':'home_team_pim'}),
    left_on=['game_id','home_team_id'], right_on=['game_id','team_id'])

# Combine PIM by Position per game with Game data by Away Team 
game_results = game_results.merge(
    game_position_pim.rename(columns={'primary_position':'away_team_position','penalty_minutes':'away_team_pim'}),
    left_on=['game_id','away_team_id','home_team_position'], right_on=['game_id','team_id','away_team_position'])

# Clean up results
game_results = game_results[['game_id','home_team','home_team_position','home_team_pim',
              'away_team','away_team_position','away_team_pim','outcome',
              'avg_pim_by_position/game_x']].rename(columns={'avg_pim_by_position/game_x':'avg_pim_by_position/game'})

In [39]:
game_results

,game_id,home_team,home_team_position,home_team_pim,away_team,away_team_position,away_team_pim,outcome,avg_pim_by_position/game
0,2017020812,Buffalo Sabres,C,0,Anaheim Ducks,C,2,away win OT,4.479150
1,2017020812,Buffalo Sabres,D,4,Anaheim Ducks,D,2,away win OT,6.016522
2,2017020812,Buffalo Sabres,W,4,Anaheim Ducks,W,2,away win OT,6.182533
3,2017020586,Anaheim Ducks,C,7,Calgary Flames,C,8,home win REG,4.479150
4,2017020586,Anaheim Ducks,D,6,Calgary Flames,D,0,home win REG,6.016522
...,...,...,...,...,...,...,...,...,...
3808,2017020704,Los Angeles Kings,D,2,Pittsburgh Penguins,D,4,away win REG,6.016522
3809,2017020704,Los Angeles Kings,W,19,Pittsburgh Penguins,W,0,away win REG,6.182533
3810,2017020726,Toronto Maple Leafs,C,0,Colorado Avalanche,C,2,away win REG,4.479150
3811,2017020726,Toronto Maple Leafs,D,0,Colorado Avalanche,D,2,away win REG,6.016522
